In [4]:
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from time import sleep

import subprocess as sp
import re

In [6]:
with open('creds.txt') as fin:
    uname, pwd = fin.read().strip().split(',')

In [8]:
def get_n_projects(headless=False):
    # create headless chrome options
    chrome_options = Options()
    if headless: chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-gpu')

    # create chrome driver instance
    driver = webdriver.Chrome(options=chrome_options)

    # navigate to the website
    driver.get("https://projectforum.tudelft.nl/")

    # click the login button
    login_button = driver.find_element(By.LINK_TEXT, "Log In (TU Delft)")
    login_button.click()

    sleep(1)
    uname_input = driver.find_element(By.ID, "username")
    pwd_input = driver.find_element(By.ID, "password")
    uname_input.send_keys(uname)
    pwd_input.send_keys(pwd)

    sleep(1)
    driver.find_element(By.ID, "submit_button").click()
    sleep(1)
    driver.find_element(By.PARTIAL_LINK_TEXT, "Software Project").click()
    sleep(1)
    driver.find_element(By.PARTIAL_LINK_TEXT, "Projects").click()

    sleep(1)
    source = re.sub(r"\s+", " ", driver.page_source).strip()

    pattern = r"The list below shows the <b>(.*)</b> projects that are available for this course"
    match = re.search(pattern, source)
    n_projects = int(match.group(1))

    # close the driver
    driver.quit()
    
    return n_projects


In [ ]:
while True:
    with open("nprojs.txt") as fin:
        n_pers = int(fin.read())

    try:
        n_read = get_n_projects(True)
    except e:
        continue
    
    if n_read != n_pers:
        assert n_pers < n_read, f"{n_pers} > {n_read}"
        sp.run(["notify-send", "--urgency=CRITICAL", f"{n_read - n_pers} project(s) added"])
    else:
        sp.run(["notify-send", f"No project(s) added"])

    with open("nprojs.txt", "w") as fout:
        print(n_read, file=fout)

    sleep(5 * 60)

    
# the scheme is as follows:
# on each iteration of the algo, check if the persisted N matches the N read.
# If not (assert N_r - N_p > 0), notify
# If equal, do nothing
    
# todo: read-eval and write-print
# + ptext format
# + minimal parsing by me

# get pers projects
# get current projects
# compare:
#  if addition: notify

In [ ]:
# BELOW CELLS NON-ESSENTIAL / TINKERING

In [ ]:
link_fmt = "https://projectforum.tudelft.nl/course_editions/59/projects?page=%s"
all_projects = dict()
for page in range(5):
    driver.get(link_fmt.format(page))
    page_projects_we = driver.find_elements(By.CLASS_NAME, "project")
    page_projects = [project_extract(we) for we in projects_we]
    for project in page_projects:
        all_projects[project['name']] = 

In [33]:
projects_we = driver.find_elements(By.CLASS_NAME, "project")
projects = [project_extract(we) for we in projects_we]

In [32]:
def project_extract(project_we):
    return {
        "name": project_we.find_element(By.CLASS_NAME, "project-name").text,
        "offerer": re.search("Offered by (.*)\n", project_we.text).group(1)
    }

{'name': 'Mathe im Advent – Making Math fun and relevant data science embedded systems',
 'offerer': 'Mathe im Leben gGmbH'}